<a href="https://colab.research.google.com/github/hikmatfarhat-ndu/veronica-thesis/blob/master/malware.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Packages

In [1]:
import tensorflow as tf 
import numpy as np 
import matplotlib.pyplot as plt
from tensorflow.keras import models,layers
from tensorflow.keras.utils import Sequence
#from tensorflow.python.keras.utils import data_utils
import math
import os
import pandas
from PIL import Image
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Conv3D,Conv2D,InputLayer,MaxPooling3D,Dropout,Flatten,MaxPooling2D,BatchNormalization,LayerNormalization


INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2


In [2]:
image_size=(256,256)
dataset=tf.keras.preprocessing.image_dataset_from_directory("data/resized/train",batch_size=32,image_size=image_size,
                                                            shuffle=True,color_mode="grayscale")
test_dataset=tf.keras.preprocessing.image_dataset_from_directory("data/resized/test",batch_size=32,image_size=image_size,
                                                            shuffle=False,color_mode="grayscale")

Found 8000 files belonging to 9 classes.
Found 868 files belonging to 9 classes.


In [3]:
def createModelYuan():
    model = Sequential()
    model.add(InputLayer(input_shape=(256,256,1)))
    #model.add(tf.keras.layers.experimental.preprocessing.Rescaling(1./255))
   
    model.add(Conv2D(64, kernel_size = (3,3),strides=(1,1), padding='same',activation = 'relu'))
    model.add(Conv2D(64, kernel_size = (3,3),strides=(1,1), padding='same',activation = 'relu'))
    model.add(MaxPooling2D(pool_size = (2,2)))

    model.add(Conv2D(128, kernel_size = (3,3),strides=(1,1), padding='same',activation = 'relu'))
    model.add(Conv2D(128, kernel_size = (3,3),strides=(1,1), padding='same',activation = 'relu'))
    model.add(MaxPooling2D(pool_size = (2,2)))
    

    model.add(Conv2D(256, kernel_size = (3,3),strides=(1,1), padding='same',activation = 'relu'))
    model.add(Conv2D(256, kernel_size = (3,3), strides=(1,1),padding='same',activation = 'relu'))
    model.add(Conv2D(256, kernel_size = (3,3), strides=(1,1),padding='same',activation = 'relu'))
    model.add(MaxPooling2D(pool_size = (2,2)))


    model.add(Conv2D(512, kernel_size = (3,3), strides=(1,1),padding='same',activation = 'relu'))
    model.add(Conv2D(512, kernel_size = (3,3), strides=(1,1),padding='same',activation = 'relu'))
    model.add(Conv2D(512, kernel_size = (3,3), strides=(1,1),padding='same',activation = 'relu'))
    model.add(MaxPooling2D(pool_size = (2,2)))

    model.add(Conv2D(512, kernel_size = (3,3), strides=(1,1),padding='same',activation = 'relu'))
    model.add(Conv2D(512, kernel_size = (3,3), strides=(1,1),padding='same',activation = 'relu'))
    model.add(Conv2D(512, kernel_size = (3,3), strides=(1,1),padding='same',activation = 'relu'))
    model.add(MaxPooling2D(pool_size = (2,2)))


    model.add(Flatten())
    model.add(Dense(1024, activation='relu'))
    model.add(Dense(9, activation = 'softmax', name = 'Output'))
    return model


In [4]:

model=createModelYuan()
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 256, 256, 64)      640       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 256, 256, 64)      36928     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 128, 128, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 128, 128, 128)     73856     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 128, 128, 128)     147584    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 64, 64, 128)       0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 64, 64, 256)       2

## Optimization

Keras can use many optimization method. In this notebook we use the __Adam__ method which can be described loosely as __adaptive__ gradient descent.

Also since the labels are __NOT__ in one_hot_encoding we use the "Sparse" version of the crossentropy loss: __SparseCategoricalCrossentropy__. Finally, if we don't specify from_logits=False then the loss function would compute softwmax before computing the loss. Since we are computing softwmax in our model already we turn this step off by specifying from_logits=False

In [5]:
# if we don't use softmax in the last layer, i.e. if the output of the
# model is NOT probabilities then use from_logits=True
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])

#history = model.fit(img_train,label_train, batch_size=128,epochs=10, 
#                    validation_data=(img_test, label_test))
#history=model.fit(train_dataset,batch_size=batch_size,epochs=3)
#model.load_weights("weights-4000.h5")
history=model.fit(dataset,epochs=3)


Epoch 1/3
 15/250 [>.............................] - ETA: 3:27 - loss: 374.2250 - accuracy: 0.1858

ResourceExhaustedError:  OOM when allocating tensor with shape[32,64,256,256] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node gradient_tape/sequential/conv2d_1/Conv2D/Conv2DBackpropInput (defined at <ipython-input-5-881d281e13c8>:11) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
 [Op:__inference_train_function_1654]

Function call stack:
train_function


In [ ]:
model.save_weights("weights-8000.h5")

### Testing the Accuracy

In [ ]:

_,test_accuracy=model.evaluate(test_dataset)